# Webscrape IRCC

In [13]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import re

### Set the URL you want to webscrape from

In [14]:
CND_SITE = 'https://www.canada.ca'
IRCC_SUFFIX='/en/immigration-refugees-citizenship/'
filter = re.compile('^' + IRCC_SUFFIX + '.*')
exclude_some = re.compile("#")
url = CND_SITE + IRCC_SUFFIX

In [19]:
def get_hrefs(url):
  response = requests.get(url)
  soup = BeautifulSoup(response.text, "html.parser")
  return set(sorted([a['href'] for a in soup.findAll('a', href=filter) if not exclude_some.search(a['href'])]))

### Connect to the site - get the first page and loop through links to get children
> #### Paramateres
>> #### SCRAPE_DEPTH: determines how many layers we loop through - less: quicker; more: more pages; setting it at 2 returns a bit over 5000 links; setting it too high make it take longer to complete
>> #### WEBCALLS_WITH_NO_BREAK: determines how often we take a one-second break from scraping. It helps to prevent from flooding the site and risk our IP be banned

In [21]:
scraped_uris = get_hrefs(url)
# in my experience two iterations returns over 5000 links
#   one can experience with more if necessary
SCRAPE_DEPTH = 1
WEBCALLS_WITH_NO_BREAK = 10

for i in range(SCRAPE_DEPTH):
  new_uris = set()
  web_calls=0
  for suffix in scraped_uris:
    new_uris |= get_hrefs(CND_SITE+suffix)
    # take a second between every few site calls to not flood the site
    web_calls += 1
    if web_calls >= WEBCALLS_WITH_NO_BREAK:
      time.sleep(1)
      web_calls = 0
  scraped_uris |= new_uris

len(scraped_uris)

629

## To download the whole data set, let's do a for loop through all a tags

In [24]:
count = 10000
for ahref in scraped_uris:
  download_url = CND_SITE + ahref
  count += 1
  urllib.request.urlretrieve(download_url,'./file-'+str(count)) 
  time.sleep(1)


In [25]:
!ls

file-10001  file-10106	file-10211  file-10316	file-10421  file-10526
file-10002  file-10107	file-10212  file-10317	file-10422  file-10527
file-10003  file-10108	file-10213  file-10318	file-10423  file-10528
file-10004  file-10109	file-10214  file-10319	file-10424  file-10529
file-10005  file-10110	file-10215  file-10320	file-10425  file-10530
file-10006  file-10111	file-10216  file-10321	file-10426  file-10531
file-10007  file-10112	file-10217  file-10322	file-10427  file-10532
file-10008  file-10113	file-10218  file-10323	file-10428  file-10533
file-10009  file-10114	file-10219  file-10324	file-10429  file-10534
file-10010  file-10115	file-10220  file-10325	file-10430  file-10535
file-10011  file-10116	file-10221  file-10326	file-10431  file-10536
file-10012  file-10117	file-10222  file-10327	file-10432  file-10537
file-10013  file-10118	file-10223  file-10328	file-10433  file-10538
file-10014  file-10119	file-10224  file-10329	file-10434  file-10539
file-10015  file-10120	file-10225 